<a href="https://colab.research.google.com/github/ammarisme/covid-19/blob/master/CV19_Synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount('/content/drive')
PATH = "/content/drive/My Drive/covid"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#set the parameters
dataset_class = "validation"
source_file =PATH+ '/input/validation/cleaned/cleaned_worldwide_cases_27_04.csv'

In [0]:
original_data = pd.read_csv(source_file)
#original_data[100:200]

In [0]:
def generate_synthetic_data(sample_dataset,run_id, window = 0, sythesize_ratio = 2):
    synthetic_data = pd.DataFrame(columns=sample_dataset.columns)
    
    for k in range(sythesize_ratio):
      if len(synthetic_data) >= len(sample_dataset) * sythesize_ratio:
        break;
      #this loop generates a set that resembles the entire dataset
      for country in sample_dataset['countryterritoryCode'].unique():
        country_synthetic = pd.DataFrame(columns=sample_dataset.columns)
        input_sequence_len = window #needs to be randomized

        #population data change
        country_data_i = sample_dataset[sample_dataset['countryterritoryCode'] == country]
        _country_code = country_data_i['_country_code'].values[0]
        #cases	deaths	popData2018	countriesAndTerritories	geoId	countryterritoryCode	continentExp	_country_code

        print(len(country_data_i))

        popData2018 = country_data_i['popData2018'].values[0]
        population_change_percentage = 1+ (np.random.normal(0,1) / 100)
        population_new = np.round(population_change_percentage*popData2018)

        country_synthetic_temp = pd.DataFrame(columns=sample_dataset.columns)

        feature_length = 3 #number of features to be randomized
        country_data_array = np.array([country_data_i['cases'].to_numpy(), country_data_i['deaths'].to_numpy(),  country_data_i['popData2018'].to_numpy()])
        country_data_array = country_data_array.reshape(feature_length,len(country_data_i))
        x = country_data_array[:3].T
            
        sets =0
        x_list = []
        dict_x = dict()
        for i in range(input_sequence_len):
          array_len = ((len(x) -i) - ((len(x)-i)%input_sequence_len))+i
          sets = int( array_len/ input_sequence_len)
          print('input seq : ', i , ' ', array_len , ' ',array_len-i , ' number of sets : ', sets)
          x_temp = x[i:array_len].T.reshape(sets,feature_length,input_sequence_len)
          uniq_keys = np.array([i+(input_sequence_len*k) for k in range(input_sequence_len)])

          x_temp = x_temp.reshape(feature_length,sets,input_sequence_len)
          
          arrays_split = np.hsplit(x_temp,sets)
          dict_x.update(dict(zip(uniq_keys, arrays_split)))

        temp_x_list  = [dict_x[i].T for i in sorted(dict_x.keys())]
        synthetic_data.append()
      

In [0]:
def verify_clean(cleaned_data):
  print(len(cleaned_data))
  print(cleaned_data.isna().any())
  print(cleaned_data.head())

In [0]:
sample_synthetic = generate_synthetic_data(original_data, run_id = "1111", window = 10, sythesize_ratio = 1)
verify_clean(sample_synthetic)

114
input seq :  0   110   110  number of sets :  11
input seq :  1   111   110  number of sets :  11
input seq :  2   112   110  number of sets :  11
input seq :  3   113   110  number of sets :  11
input seq :  4   114   110  number of sets :  11
input seq :  5   105   100  number of sets :  10
input seq :  6   106   100  number of sets :  10
input seq :  7   107   100  number of sets :  10
input seq :  8   108   100  number of sets :  10
input seq :  9   109   100  number of sets :  10
(100, 10, 1, 3)


TypeError: ignored

In [0]:
#testing for accuracy
def verify_synthetic_data(synthetic_data,original_data, print_results = False):
  countryterritoryCode ='CN'
  #test_synthetic_data = synthetic_data[synthetic_data['countryterritoryCode'] == countryterritoryCode]
  test_synthetic_data = synthetic_data
  control_set = original_data #[original_data['countryterritoryCode']==countryterritoryCode]
  case_mean = np.mean(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy())
  death_mean = np.mean(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy())
  pop_mean = np.mean(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy())

  case_std = np.std(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy())
  death_std = np.std(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy())
  pop_std = np.std(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy())

  if print_results:
    print(test_synthetic_data.head())
    print(control_set.head())
    print('original data length : ', len(control_set))
    print('synthetic data length : ',len(test_synthetic_data))
    print('min sequence length : ',min([len(test_synthetic_data[test_synthetic_data['countryterritoryCode']==country]) for country in test_synthetic_data['countryterritoryCode'].unique()]))
    print('max sequence length : ',max([len(test_synthetic_data[test_synthetic_data['countryterritoryCode']==country]) for country in test_synthetic_data['countryterritoryCode'].unique()]))

    print('Mean diff. cases : ', np.mean(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy()))
    print('Mean diff. deaths : ', np.mean(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy()))
    print('Mean diff. population : ', np.mean(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy()))
  
  return  [case_mean, death_mean, pop_mean, case_std , death_std , pop_std ]

In [0]:
""" results = []
 for i in range(2,20):
   syn_data = generate_synthetic_data(original_data,i, sythesize_ratio = 1)
   print('window :', i, ' len :', len(syn_data))
   results.append(verify_synthetic_data(syn_data,original_data))

"""

" results = []\n for i in range(2,20):\n   syn_data = generate_synthetic_data(original_data,i, sythesize_ratio = 1)\n   print('window :', i, ' len :', len(syn_data))\n   results.append(verify_synthetic_data(syn_data,original_data))\n\n"

In [0]:
run_id = 3042013 #ddmyyhh
for i in range(1):
  rand_number = np.random.normal(0,1)
  if rand_number >= 0:
    rand_number =1
  else:
    rand_number =0
  window_size = rand_number*(15 + np.round(np.random.normal(0,1))) + (1 - rand_number)*(4 + np.round(np.random.normal(0,1)))
  synthetic_data = generate_synthetic_data(original_data, run_id = str(i)+str(run_id), window = int(window_size), sythesize_ratio = 1)
  synthetic_data.to_csv(PATH+'/input/raw/synthetic/synthetic_'+str(i)+'_'+str(run_id)+'_dt27_04_'+str(int(window_size))+'.csv',  index=False )
  print('synth step : ', i, ' len : ', len(synthetic_data))

synth step :  0  len :  7552


In [0]:
run_id = 2842001 #ddmyyhh


synth step :  27  len :  75520


KeyboardInterrupt: ignored